In [1]:
import geopandas as gpd
import numpy as np
from rasterstats import zonal_stats
from osgeo import gdal
from sklearn.metrics import confusion_matrix, classification_report

# 1. Chargement des données
vector_path = "../results/data/sample/Sample_BD_foret_T31TCJ.shp"
gdf = gpd.read_file(vector_path)

# Charger le raster avec GDAL
raster_path = "../results/data/classif/carte_essences_echelle_pixel.tif"
raster = gdal.Open(raster_path)

/opt/conda/lib/python3.12/site-packages/osgeo/gdal.py:311: FutureWarning: Neither gdal.UseExceptions() nor gdal.DontUseExceptions() has been explicitly called. In GDAL 4.0, exceptions will be enabled by default.
  warnings.warn(


In [2]:
# 2. Calcul de la surface des polygones en hectares
gdf["Surface_ha"] = gdf.geometry.area / 10000  # Conversion de m² en hectares

In [3]:
# 3. Calcul des statistiques zonales
stats = zonal_stats(
    vector_path,
    raster_path,
    categorical=True,  # Permet de calculer la distribution des classes
    geojson_out=True
)

# Ajouter les pourcentages calculés à la GeoDataFrame
for i, stat in enumerate(stats):
    for key, value in stat["properties"].items():
        if isinstance(key, str) and key.startswith("cat_"):  # Vérifier que la clé est une chaîne de caractères
            gdf.loc[i, key] = value

In [4]:
# 4. Implémentation des règles de décision
def arbre_decision(surface, df):
    # Définir les listes des types de feuillus et de conifères
    feuillus_types = ['Autre feuillus', 'Chêne', 'Robinier', 'Peupleraie']
    coniferes_types = ['Autres conifères', 'Autres pin', 'Douglas', 'Pin laricio', 'Pin noir', 'Pin maritime']

    # Compter les occurrences de chaque type d'arbre dans la colonne Nom_pixel
    total_pixels = len(df)
    feuillus_count = df['Nom_pixel'].isin(feuillus_types).sum()
    coniferes_count = df['Nom_pixel'].isin(coniferes_types).sum()

    # Calculer les pourcentages des feuillus et des conifères
    somme_feuillus = (feuillus_count / total_pixels) * 100
    somme_coniferes = (coniferes_count / total_pixels) * 100

    # Règle 1 : Surface < 2ha ?
    if surface < 2:
        # Règle 2a : Somme des feuillus > 75%
        if somme_feuillus > 75:
            # Règle 3aa : Feuillus en îlots
            return "Feuillus en îlots"
        else:
            # Règle 3ab : Somme des conifères > 75% ?
            if somme_coniferes > 75:
                # Règle 4aba : Conifères en îlots
                return "Conifères en îlots"
            else:
                # Règle 4abb : Somme des conifères > Somme des feuillus ?
                if somme_coniferes > somme_feuillus:
                    return "Mélange feuillus"
                else:
                    # Règle 4bba : Mélange feuillus
                    return "Mélange feuillus"
    else:
        # Règle 2b : Classe C > 75% ?
        cat_1 = df.get("cat_1", 0)  # Classe 1
        if cat_1 > 75:
            # Règle 3ba : Polygone en classe C
            return "Polygone en classe C"
        else:
            # Règle 3ab : Somme des conifères > 75% ?
            if somme_coniferes > 75:
                # Règle 4bba : Mélange feuillus
                return "Mélange feuillus"
            else:
                # Règle 4bb : Somme des feuillus > 75% ?
                if somme_feuillus > 75:
                    # Règle 5abba : Mélange de conifères prépondérants et feuillus
                    return "Mélange de conifères prépondérants et feuillus"
                else:
                    # Règle 5abbb : Mélange de feuillus prépondérants et conifères
                    return "Mélange de feuillus prépondérants et conifères"

    # Règle 6bbbba : Mélange de conifères prépondérants et feuillus
    return "Mélange de conifères prépondérants et feuillus"

In [5]:
# Fonction de classification qui applique les règles de décision
def classify_peuplement(row):
    surface_ha = row["Surface_ha"]
    
    # Extrait le DataFrame correspondant à chaque polygone pour les statistiques zonales
    stats_df = gdf[gdf["geometry"] == row["geometry"]]

    # Appeler l'arb
    return arbre_decision(surface_ha, stats_df)

# Appliquer la classification à chaque ligne
gdf["code_predit"] = gdf.apply(classify_peuplement, axis=1)

In [6]:
# Étape 1 : Vérifier les champs Nom_pixel et Nom_objet
print("Analyse initiale :")
print(f"Nombre de valeurs NaN dans Nom_pixel : {gdf['Nom_pixel'].isna().sum()}")
print(f"Nombre de valeurs NaN dans Nom_objet : {gdf['Nom_objet'].isna().sum()}")

# Étape 2 : Filtrer les lignes où Nom_pixel est vide
gdf_vide = gdf[gdf["Nom_pixel"].isna()]

# Étape 3 : Comparer les prédictions (code_predit) avec Nom_objet
gdf_vide["Correspondance"] = gdf_vide["code_predit"] == gdf_vide["Nom_objet"]

# Analyse des erreurs
erreurs = gdf_vide[gdf_vide["Correspondance"] == False]
print("Lignes avec des erreurs de correspondance entre code_predit et Nom_objet :")

# Étape 5 : Vérifier les prédictions
y_true = gdf["Code_pixel"]
y_pred = gdf["code_predit"]

# Vérifier les types pour éviter des problèmes dans sklearn
if y_true.dtype != y_pred.dtype:
    print("Les types de y_true et y_pred diffèrent. Conversion nécessaire.")
    y_true = y_true.astype(str)
    y_pred = y_pred.astype(str)

# Calculer la matrice de confusion et le rapport de classification
conf_matrix = confusion_matrix(y_true, y_pred, labels=np.unique(y_true))
report = classification_report(y_true, y_pred)

print("Matrice de confusion :")
print(conf_matrix)

print("\nRapport de classification :")
print(report)



Analyse initiale :
Nombre de valeurs NaN dans Nom_pixel : 8303
Nombre de valeurs NaN dans Nom_objet : 0
Lignes avec des erreurs de correspondance entre code_predit et Nom_objet :
Les types de y_true et y_pred diffèrent. Conversion nécessaire.


/opt/conda/lib/python3.12/site-packages/geopandas/geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()

Matrice de confusion :
[[0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]]

Rapport de classification :
                                                precision    recall  f1-score   support

                                          11.0       0.00      0.00      0.00     128.0
                                          12.0       0.00      0.00      0.00    2172.0
                                          13.0       0.00      0.00      0.00      56.0
                                          14.0       0.00      0.00      0.00     172.0
                                          21.0       0.00      0.00      0.00       5.0
                                          22.0       0.00      0.00      0.00       8.0
                                          23.0       0.00      0.00      0.00      18.0
            

/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_